https://www.tensorflow.org/responsible_ai/privacy/tutorials/privacy_report

https://github.com/tensorflow/privacy/tree/master/tensorflow_privacy/privacy/privacy_tests/membership_inference_attack

Here I train a new approximant layer as substitute for final sigmoid layer  that is trained on input-output dataset of the output softmax layer of original model. This trained new approximant layer is than substituted in the original model to form refined model
The output vec is one hot encoded output of layer

The refined model is trained again with loss as MSE.

I train this reference layer using MSE Loss

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tensorflow import keras
import tensorflow as tf

# Define the neural network architecture
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.reshape=nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.nonlinear = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.reshape(x)
        # x = torch.flatten(x, start_dim=1)
        # x = x.reshape(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x= self.nonlinear(x)
        # x = torch.softmax(x, dim=1)
        return x

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_data = datasets.MNIST('data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('data', train=False, transform=transform)

train_loader = DataLoader(train_data, batch_size=64*20, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64*20, shuffle=False)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MNISTClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 1
for epoch in range(epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()


2023-04-19 08:27:57.242918: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 08:27:58.064712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model.to('cpu')

MNISTClassifier(
  (reshape): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (nonlinear): Softmax(dim=1)
)

In [3]:
from sklearn.metrics import accuracy_score
import numpy as np
in_preds = []
in_label = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        # inputs = inputs.to(device)
        outputs = model(inputs)
        #load outputs to cpu
        outputs = outputs.cpu()
        in_preds.append(outputs)
        in_label.append(labels)
    in_preds = torch.cat(in_preds)
    in_label = torch.cat(in_label)
print(
    "Test Accuracy before Linearization is: ",
    accuracy_score(np.array(torch.argmax(in_preds, axis=1)), np.array(in_label)),
)

Test Accuracy before Linearization is:  0.8816


In [4]:
# count the number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 109,386 trainable parameters


In [5]:
model.to(device)

MNISTClassifier(
  (reshape): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (nonlinear): Softmax(dim=1)
)

In [6]:
# create dataset for linearization
in_vec=[]
out_vec=[]

second_last_output = None

def hook(module, input, output):
    global second_last_output
    second_last_output = output

# Register the hook to the second last layer (e.g., the `fc` layer in this example)
handle = model.fc3.register_forward_hook(hook)


for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    output = model(images)
    pre_output = second_last_output
    np_output = output.cpu().detach().numpy()
    np_pre_output = pre_output.cpu().detach().numpy()
    # cnvert to torch
    np_output = torch.from_numpy(np_output)
    np_pre_output = torch.from_numpy(np_pre_output)
    in_vec.append(np_output)
    out_vec.append(np_pre_output)

in_vec = torch.cat(in_vec)
out_vec = torch.cat(out_vec)

handle.remove()

model.to('cpu')

MNISTClassifier(
  (reshape): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (nonlinear): Softmax(dim=1)
)

In [7]:
from torch.utils.data import TensorDataset, DataLoader


# Create a PyTorch dataset
layer_dataset = TensorDataset(in_vec, out_vec)

# Create a PyTorch DataLoader
batch_size = 64*20  # or any other batch size you prefer
layer_train_loader = DataLoader(layer_dataset, batch_size=batch_size, shuffle=True)


# Define the neural network architecture
class Approximant(nn.Module):
    def __init__(self):
        super(Approximant, self).__init__()
        self.fc1 = nn.Linear(10, 10)
        # self.minnonlinear = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        # x = self.minnonlinear(x)
        return x



linearized_proxy=Approximant().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(linearized_proxy.parameters(), lr=1e-3)



epochs = 1


# train the proxy linear layer
for epoch in range(epochs):
    for inputs, outputs in layer_train_loader:
        inputs, outputs = inputs.to(device), outputs.to(device)
        optimizer.zero_grad()
        pred_outputs = linearized_proxy(inputs)
        loss = criterion(pred_outputs.to(device),outputs.to(device))
        loss.backward()
        optimizer.step()

linearized_proxy.to('cpu')


Approximant(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
)

In [8]:
class CombinedModel(nn.Module):
    def __init__(self, model1, model2):
        super().__init__()
        self.model1 = model1
        self.model2 = model2

    def forward(self, x):
        x1= self.model1(x)
        x = self.model2(x1)
        return x


class StackedModel(nn.Module):
    def __init__(self, model1, model2):
        super(StackedModel, self).__init__()
        self.model1_layers = model1.children()
        self.model2_layers = model2.children()
        
        # Concatenate layers from both models
        stacked_layers = []
        for layer in self.model1_layers:
            stacked_layers.append(layer)
        for layer in self.model2_layers:
            stacked_layers.append(layer)
        
        self.stacked_model = nn.Sequential(*stacked_layers)

    def forward(self, x):
        return self.stacked_model(x)



# The proxy linear model
model2 = linearized_proxy

# Remove the last layer of the original model
model1 = nn.Sequential(*list(model.children())[:-1])


# Combine the amputed original model and proxy linear model

# combined_model = CombinedModel(model1, model2)
combined_model = StackedModel(model1, model2)


combined_model.to(device)



StackedModel(
  (stacked_model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [9]:
#------------------------------train-------------------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(combined_model.parameters(), lr=1e-5)

epochs = 1

# train the proxy linear layer
for epoch in range(epochs):
    for inputs, outputs in train_loader:
        #One hot encode outputs
        outputs = torch.nn.functional.one_hot(outputs, num_classes=10).float()
        inputs, outputs = inputs.to(device), outputs.to(device)
        optimizer.zero_grad()
        pred_outputs = combined_model(inputs)
        #convert to float
        pred_outputs = pred_outputs.float()

        loss = criterion(pred_outputs.to(device),outputs)
        loss.backward()
        optimizer.step()
#------------------------------------------------------------------


# count the number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The modified model has {count_parameters(combined_model):,} trainable parameters')


from sklearn.metrics import accuracy_score
import numpy as np


in_preds = []
in_label = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        outputs = combined_model(inputs)
        #load outputs to cpu
        outputs = outputs.detach().cpu()
        in_preds.append(outputs)
        in_label.append(labels)
    in_preds = torch.cat(in_preds)
    in_label = torch.cat(in_label)
print(
    "Test Accuracy after Linearization is: ",
    accuracy_score(np.array(torch.argmax(in_preds, axis=1)), np.array(in_label)),
)

The modified model has 109,496 trainable parameters
Test Accuracy after Linearization is:  0.176


In [10]:
target_model = combined_model.to('cpu')

In [11]:
target_model.eval()

StackedModel(
  (stacked_model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [12]:
import time
input_tensor=torch.randn((1, 1, 28, 28))

model.eval()
# Perform a warm-up run to avoid potential overhead caused by initial device setup
with torch.no_grad():
    _ = target_model(input_tensor)



num_iterations = 1000  # Choose a suitable number of iterations to average over

start_time = time.time()
with torch.no_grad():
    for _ in range(num_iterations):
        _ = target_model(input_tensor)
end_time = time.time()

inference_latency = (end_time - start_time) / num_iterations
#convert to microseconds
inference_latency = inference_latency * 1000000
print(f'Inference latency: {inference_latency:.20f} micro-seconds')



Inference latency: 76.68638229370117187500 micro-seconds
